Whats' released?

# Setup

In [1]:
from __future__ import print_function

In [2]:
import sys
import os
from IPython.display import clear_output
import RDF
import pandas

In [3]:
from curation_common import *

In [4]:
from htsworkflow.util.rdfhelp import fromTypedNode

In [5]:
%load_ext rdfmagic

The rdfmagic extension is already loaded. To reload it, use:
  %reload_ext rdfmagic


In [6]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [7]:
model = get_model(use_contexts=False)

# Load status from DCC

In [8]:
search = server.get_json('/search/?type=experiment&lab.title=Barbara Wold, Caltech&award.rfa=ENCODE3')
len(search['@graph'])

196

In [9]:
for i, row in enumerate(search['@graph']):
    experiment_id = row['@id']
    #clear_output()
    print('loading', experiment_id, i, len(search['@graph']))
    experiment = server.get_jsonld(experiment_id)
    load_jsonld_into_model(model, experiment)

loading /experiments/ENCSR822EWO/ 0 196
loading /experiments/ENCSR805QYO/ 1 196
loading /experiments/ENCSR734SOQ/ 2 196
loading /experiments/ENCSR909YEQ/ 3 196
loading /experiments/ENCSR298YHA/ 4 196
loading /experiments/ENCSR220HFQ/ 5 196
loading /experiments/ENCSR650MMK/ 6 196
loading /experiments/ENCSR953TKB/ 7 196
loading /experiments/ENCSR295UJP/ 8 196
loading /experiments/ENCSR323FHJ/ 9 196
loading /experiments/ENCSR195BCE/ 10 196
loading /experiments/ENCSR782WIL/ 11 196
loading /experiments/ENCSR138VRG/ 12 196
loading /experiments/ENCSR390KQJ/ 13 196
loading /experiments/ENCSR427FOY/ 14 196
loading /experiments/ENCSR846FYI/ 15 196
loading /experiments/ENCSR927IOO/ 16 196
loading /experiments/ENCSR739YUO/ 17 196
loading /experiments/ENCSR100KRK/ 18 196
loading /experiments/ENCSR198EIH/ 19 196
loading /experiments/ENCSR746LUP/ 20 196
loading /experiments/ENCSR388ZNJ/ 21 196
loading /experiments/ENCSR331XCE/ 22 196
loading /experiments/ENCSR178GUS/ 23 196
loading /experiments/ENCSR

# Current ENCODE3 Experiment Status

In [10]:
%%sparql -m model -c
select ?accession ?assay_name ?description ?status ?release
where {
  ?accession a experiment:experiment ;
             rdf:description ?description ;
             experiment:assay_term_name ?assay_name ;
             experiment:status ?status .
  OPTIONAL { ?accession experiment:date_released ?release .}
}
order by ?status ?description

Found 196 rows.


accession,assay_name,description,status,release
experiments:ENCSR516XOY/,RNA-seq,RNA-seq on human Betz cell obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR894SKI/,RNA-seq,RNA-seq on human Meynert cell (solitary pyramidal neuron humans in visual cortex) obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR643FBX/,RNA-seq,RNA-seq on human brain stem motor neuron human obtained with laser capture micro dissection human,preliminary,None
experiments:ENCSR784SMP/,RNA-seq,RNA-seq on human brush border cell of the proximal tubule obtained with laser capture micro dissection human,preliminary,None
experiments:ENCSR015EJF/,RNA-seq,RNA-seq on human cholangiocyte obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR582YQI/,RNA-seq,RNA-seq on human endothelial cell of hepatic sinusoid obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR369OAV/,RNA-seq,RNA-seq on human glomerular visceral epithelial cell obtained with laser capture micro dissection human,preliminary,None
experiments:ENCSR358IBS/,RNA-seq,RNA-seq on human hepatic natural killer cells obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR528DTF/,RNA-seq,RNA-seq on human hepatic oval stem cell obtained with histology guided laser capture micro dissection human,preliminary,None
experiments:ENCSR792DJY/,RNA-seq,RNA-seq on human hepatic stellate cell obtained with histology guided laser capture micro dissection human,preliminary,None


# Current Encode3 Library status

In [11]:
%%sparql -m model
select ?p ?o
where {
    <https://www.encodeproject.org/organisms/mouse/> ?p ?o
}

p,o
rdf:type,https://www.encodeproject.org/profiles/organism.json#organism
rdf:type,https://www.encodeproject.org/profiles/organism.json#item
https://www.encodeproject.org/profiles/organism.json#name,mouse
https://www.encodeproject.org/profiles/organism.json#schema_version,2
https://www.encodeproject.org/profiles/organism.json#scientific_name,Mus musculus
https://www.encodeproject.org/profiles/organism.json#status,current
https://www.encodeproject.org/profiles/organism.json#taxon_id,10090
https://www.encodeproject.org/profiles/organism.json#uuid,3413218c-3d86-498b-a0a2-9a406638e786


In [12]:
%%sparql -m model -c -o biosamples
select ?biosample ?organism ?name
where {
    ?biosample a biosample:biosample ;
               biosample:organism ?organism .
    ?organism a <https://www.encodeproject.org/profiles/organism.json#organism> ;
              <https://www.encodeproject.org/profiles/organism.json#name> ?name .
}

Found 241 rows.


In [13]:
biosample_map = {
    str(row['biosample']): str(row['name']) for row in biosamples
}

#for row in biosamples:
#    bio = str(row['biosample'])
#    name = str(row['name'])
#    biosample_map[bio] = name

In [ ]:
%%sparql -m model -c -o library_status
select ?experiment ?library ?biosample ?description ?alias ?status ?created ?date_released
where {
  ?experiment a experiment:experiment ;
             rdf:description ?description ;
             experiment:status ?status ;
             experiment:replicates ?replicate .
   OPTIONAL { ?experiment experiment:date_released ?date_released .} 
   ?replicate replicate:library ?library .
   ?library library:date_created ?created ;
            library:biosample ?biosample .
   
   OPTIONAL { ?library library:aliases ?alias . }
}
order by ?created ?experiment

In [ ]:
library_

In [15]:
records = []
for row in library_status:
    record = { k: str(row[k]) for k in row }
    record['alias'] = record['alias'].replace('barbara-wold:', '')
    record['created'] = record['created'][:10]
    record['organism'] = biosample_map.get(record['biosample'], None)
    records.append(record)
librarydf = pandas.DataFrame(records, 
                             columns=['experiment', 'library', 'alias', 'organism', 'description', 'status', 'created'],
                             )

In [16]:
librarydf.to_excel('/tmp/library.xlsx', index=False)

In [17]:
librarydf.to_excel?

# Current Encode 3 File status

In [18]:
%%sparql -m model -c -o file_status
select distinct ?experiment ?description ?status ?created
where {
  ?experiment a experiment:experiment ;
             rdf:description ?description ;
             experiment:status ?status ;
             experiment:files ?file .
   ?file file:date_created ?created ;
         file:status ?status .
}
order by ?created ?status

Found 931 rows.


In [19]:
%%sparql -m model
select ?p ?o 
where {
  <https://www.encodeproject.org/libraries/ENCLB646JVH/> ?p ?o.
}

p,o
rdf:type,library:library
rdf:type,library:item
library:accession,ENCLB646JVH
library:aliases,barbara-wold:13446
library:award,https://www.encodeproject.org/awards/U54HG006998/
library:biosample,https://www.encodeproject.org/biosamples/ENCBS616ZXC/
library:date_created,2015-06-26T00:12:00.841890+00:00
library:documents,https://www.encodeproject.org/documents/49fd5b3b-9878-4ec8-92bf-ef6dce9116e3/
library:extraction_method,n/a
library:fragmentation_method,chemical (Nextera tagmentation)


In [23]:
%%sparql -m model
select ?p ?o 
where {
    <https://www.encodeproject.org/experiments/ENCSR950BNG/> ?p ?o
}

p,o
rdf:type,experiment:experiment
rdf:type,experiment:dataset
rdf:type,experiment:item
rdf:description,Total RNA-seq on postnatal 0 day mouse liver (13pgs)
experiment:accession,ENCSR950BNG
experiment:assay_synonyms,transcription profiling by high throughput sequencing
experiment:assay_synonyms,RNA-seq assay
experiment:assay_term_id,http://purl.obolibrary.org/obo/OBI_0001271
experiment:assay_term_name,RNA-seq
experiment:assembly,mm10


In [21]:
%%sparql -m model
select ?p ?o
where {
    <https://www.encodeproject.org/replicates/a37a8cff-5bf7-47fc-b9a0-fa2aafebb9c3/> ?p ?o .
}

p,o
rdf:type,replicate:replicate
rdf:type,replicate:item
replicate:biological_replicate_number,1
replicate:date_created,2015-06-26T00:17:47.115331+00:00
replicate:experiment,experiments:ENCSR388ZNJ/
replicate:library,libraries:ENCLB869UHR/
replicate:notes,{}
replicate:schema_version,5
replicate:status,in progress
replicate:submitted_by,https://www.encodeproject.org/users/bc5b62f7-ce28-4a1e-b6b3-81c9c5a86d7a/


In [22]:
%%sparql -m model
select ?p ?o 
where {
  <https://www.encodeproject.org/biosamples/ENCBS616ZXC/> ?p ?o
    }

p,o
rdf:type,biosample:biosample
rdf:type,biosample:item
rdf:description,LCMD-504_Mm_Castaneus_x_C57Bl-granule-layer 30 purkinje cell area equivalent
biosample:accession,ENCBS616ZXC
biosample:age,14.0
biosample:age_display,14.0 month
biosample:age_units,month
biosample:aliases,barbara-wold:sample-15346
biosample:award,https://www.encodeproject.org/awards/U54HG006998/
biosample:biosample_term_id,UBERON:0002956
